In [1]:
import os

In [2]:
pwd

'c:\\Users\\DELL\\Documents\\End_to_End_machine_learning_project_with_mlflow\\research'

In [3]:
os.chdir("..")

In [4]:
pwd

'c:\\Users\\DELL\\Documents\\End_to_End_machine_learning_project_with_mlflow'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_url:str
    local_data_file:Path
    unzip_dir:str

In [6]:

from mlproject.utils.common import read_yaml,create_directories

In [7]:
# Define constants RIGHT HERE - guaranteed to work
CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")
SCHEMA_FILE_PATH = Path("schema.yaml")


In [8]:
#creating a class configurationManager

class ConfigurationManager():
    def __init__(self,config_path=CONFIG_FILE_PATH,params_path=PARAMS_FILE_PATH,schema_path=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_path)
        self.params=read_yaml(params_path)
        self.schema=read_yaml(schema_path)

        create_directories([self.config.artifacts_root])

    def get_ingestion_configuration(self):
        self.data_ingestion=self.config.data_ingestion
        create_directories([self.data_ingestion.root_dir])

        #return an instance of the dataingestionclass

        data_ingestion_config= DataIngestionConfig(
            root_dir=Path(self.data_ingestion.root_dir),
            source_url=self.data_ingestion.source_url,
            local_data_file=Path(self.data_ingestion.local_data_file),
            unzip_dir=self.data_ingestion.unzip_dir

        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from mlproject import logger
from mlproject.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self,config=DataIngestionConfig):
        self.config=config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [11]:
try:
    config=ConfigurationManager()
    data_ingestion_configuration=config.get_ingestion_configuration()
    data_ingestion=DataIngestion(config=data_ingestion_configuration)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-10-30 06:24:27,216: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-30 06:24:27,222: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-30 06:24:27,231: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-10-30 06:24:27,233: INFO: common: created directory at: artifacts]
[2025-10-30 06:24:27,238: INFO: common: created directory at: artifacts/data_ingestion]
[2025-10-30 06:24:29,120: INFO: 505601528: artifacts\data_ingestion\data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6EDC:B938A:432166B:473E3C7:69024DEA
Accept-Ranges: bytes
Date: Wed, 29

In [12]:
from pathlib import Path

config_path = Path("config/config.yaml")
print(f"File exists: {config_path.exists()}")
print(f"File size: {config_path.stat().st_size} bytes")  # If 0, file is empty

File exists: True
File size: 273 bytes
